# LLM 调用

1. LLM 调用

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage,AIMessage
import os

os.environ["OPENAI_API_KEY"] = "sk-xxx"
os.environ["OPENAI_API_BASE"] = "https://api.deepseek.com"

openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_base = os.getenv("OPENAI_API_BASE")

chat = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    openai_api_key = openai_api_key,
    openai_api_base = openai_api_key
)

messages = [
    AIMessage(role="system",content="你好，我是tomie！"),
    HumanMessage(role="user",content="你好tomie，我是狗剩!"),
    AIMessage(role="system",content="认识你很高兴!"),
    HumanMessage(role="user",content="你知道我叫什么吗？")
]

response = chat.invoke(messages)
print(response)

2. 流式调用

In [ ]:
# LLM类大模型的流式输出方法
from langchain.llms import OpenAI
import os
api_base = os.getenv("OPENAI_PROXY")
api_key = os.getenv("OPENAI_API_KEY")

#构造一个llm
llm = OpenAI(
    model = "gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key = api_key,
    openai_api_base = api_base,
    max_tokens=512,
)

for chunk in llm.stream("写一首关于秋天的诗歌"):
    print(chunk,end="",flush=False)

3. Token 追踪

In [ ]:
# LLM的toekn追踪
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
import os
api_base = os.getenv("OPENAI_PROXY")
api_key = os.getenv("OPENAI_API_KEY")

#构造一个llm
llm = OpenAI(
    model = "gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key = api_key,
    openai_api_base = api_base,
    max_tokens=512,
)

# 通过 callback 来驱动
with get_openai_callback() as cb:
    result = llm.invoke("给我讲一个笑话")
    print(result)
    print(cb)

In [ ]:
# chatmodels的token追踪
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
import os
api_base = os.getenv("OPENAI_PROXY")
api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    model = "gpt-4",
    temperature=0,
    openai_api_key = api_key,
    openai_api_base = api_base,
    max_tokens=512,
)

with get_openai_callback() as cb:
    result = llm.invoke("给我讲一个笑话")
    print(result)
    print(cb)

4. 自定义输出

- 输出函数参数
- 输出 json
- 输出 list
- 输出日期

In [ ]:
#讲笑话机器人：希望每次根据指令，可以输出一个这样的笑话(小明是怎么死的？笨死的)

from langchain.llms import  OpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel,Field,validator
from typing import  List
import os
api_base = os.getenv("OPENAI_PROXY")
api_key = os.getenv("OPENAI_API_KEY")

# 构造LLM
model = OpenAI(
    model = "gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key = api_key,
    openai_api_base = api_base,
)

# 定义个数据模型，用来描述最终的实例结构
class Joke(BaseModel):
    setup:str = Field(description="设置笑话的问题")
    punchline:str = Field(description="回答笑话的答案")

    #验证问题是否符合要求
    @validator("setup")
    def question_mark(cls,field):
        if field[-1] != "？":
            raise ValueError("不符合预期的问题格式!")
        return field

# 将Joke数据模型传入
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template = "回答用户的输入.\n{format_instructions}\n{query}\n",
    input_variables = ["query"],
    partial_variables = {"format_instructions":parser.get_format_instructions()}
)

prompt_and_model = prompt | model # 管道操作符
out_put = prompt_and_model.invoke({"query":"给我讲一个笑话"})

print("out_put:",out_put)
# out_put: {"setup": "为什么猫咪总是喜欢把东西丢到地上？", "punchline": "因为它们觉得地球是圆的，所以才会有东西掉下来。"}

parser.invoke(out_put)
# Joke(setup='为什么猫咪总是喜欢把东西丢到地上？', punchline='因为它们觉得地球是圆的，所以才会有东西掉下来。')


In [ ]:
#LLM的输出格式化成python list形式，类似['a','b','c']

from langchain.output_parsers import  CommaSeparatedListOutputParser
from langchain.prompts import  PromptTemplate
from langchain.llms import OpenAI
import os
api_base = os.getenv("OPENAI_PROXY")
api_key = os.getenv("OPENAI_API_KEY")

#构造LLM
model = OpenAI(
    model = "gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key = api_key,
    openai_api_base = api_base,
)

parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate(
    template = "列出5个{subject}.\n{format_instructions}",
    input_variables = ["subject"],
    partial_variables = {"format_instructions":parser.get_format_instructions()}
)

_input = prompt.format(subject="常见的小狗的名字")
output = model(_input)
print(output)

parser.parse(output)
# ['1. 小白 (Xiao Bai)\n2. 小黑 (Xiao Hei)\n3. 小贝 (Xiao Bei)\n4. 小旺 (Xiao Wang)\n5. 小强 (Xiao Qiang)']